In [1]:
from nltk.stem import SnowballStemmer
from yaml import load, SafeLoader
import string

In [2]:
snowball_stemmer = SnowballStemmer('russian')

In [3]:
snowball_stemmer.stem('благодарны')

'благодарн'

In [4]:
for word in ['благодарность', 'благодарностью', 'благодарны']:
    print(snowball_stemmer.stem(word))

благодарн
благодарн
благодарн


In [5]:
params = load(open('data/params.yaml', mode='r', encoding='utf-8'), Loader=SafeLoader)

In [6]:
params

{'positive': ['благодарн',
  'хорош',
  'довольн',
  'качествен',
  'отличн',
  'понрав'],
 'negative': ['ужасн', 'отвратительн']}

In [7]:
def clear_punctuation(text):
    """Удаление знаков пунктуации из текста текст"""
    symbols = '.,!()"<>'
    spaces = ' ' * len(symbols)
    return text.translate(text.maketrans(symbols, spaces))

In [8]:
text = 'Просто шикарный клуб! Ходили с другом на "Animal Джаz"! Остались очень довольны, атмосфера очень уютная, дружелюбная, есть второй этаж, бар'

In [9]:
text_no_puctuation = clear_punctuation(text)
positive_words_list = [x for x in text_no_puctuation.split(' ') if snowball_stemmer.stem(x) in params['positive']]

In [10]:
positive_words_list

['довольны']

In [11]:
positive_word_count = len([x for x in text_no_puctuation.split(' ') if snowball_stemmer.stem(x) in params['positive']])

In [12]:
positive_word_count

1

In [13]:
def classifier(text):
    """Классификация отзыва text на 'positive', 'negative' и 'undef' по совпадающим основам слов из params.yaml"""
    text = clear_punctuation(text)
    positive_word_count = len([x for x in text.split(' ') if snowball_stemmer.stem(x) in params['positive']])
    negative_words_count = len([x for x in text.split(' ') if snowball_stemmer.stem(x) in params['negative']])
    if positive_word_count > negative_words_count:
        return 'positive'
    elif positive_word_count < negative_words_count:
        return 'negative'
    return 'undef'

In [14]:
text = 'Просто шикарный клуб! Ходили с другом на "Animal Джаz"! Остались очень довольны, атмосфера очень уютная, дружелюбная, есть второй этаж, бар'

In [15]:
classifier(text)

'positive'

In [16]:
text = 'Ужасное место. Сотрудники клуба от них в восторге! А культурным людям тут не место.'

In [17]:
classifier(text)

'negative'

In [18]:
f = open('data/texts_opinions.txt', mode='r', encoding='utf-8')
f_classified = open('data/textclassified.txt', mode='w', encoding='utf8')
for line in f:
    line = line.strip()
    f_classified.write(f'{classifier(line)}\n')
f.close()
f_classified.close()

In [20]:
f_classified = open('data/textclassified.txt', mode='r', encoding='utf8')
f_ratings = open('data/texts_ratings.txt', mode = 'r', encoding = 'utf-8')
classified_list = [line.strip() for line in f_classified]
ratings_list = [line.strip() for line in f_ratings]
right_classifications = 0
total_defined_ratings = 0
for i in range(len(classified_list)):
    if classified_list[i] != 'undef':
        total_defined_ratings += 1
        if classified_list[i] == ratings_list[i]:
            right_classifications += 1
print('Доля верно классифицированных отзывов: {:.0%}'.format(right_classifications / total_defined_ratings))
f_classified.close()
f_ratings.close()

Доля верно классифицированных отзывов: 77%
